In [2]:
!pip3 -q install tensorflow==2.1.0 tensorflow-text==2.1.1 tensorflow-hub==0.7.0 tensorflow-datasets

In [1]:
!ulimit -n 1024

In [2]:
from textrank import USETextRank
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [ ]:
ds_valid = tfds.load(name="cnn_dailymail", split="valid", shuffle_files=True)